This code calculates and stores cosmological power spectra 
and related coefficients for weak lensing and matter power spectra based on cosmological models.

In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '../LimberCloud/' #'/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

Choosing a cosmological model.

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY_INFO = json.load(file)

COSMOLOGY = pyccl.Cosmology(
    h = COSMOLOGY_INFO['H'],
    w0 = COSMOLOGY_INFO['W0'],
    wa = COSMOLOGY_INFO['WA'], 
    n_s = COSMOLOGY_INFO['NS'], 
    A_s = COSMOLOGY_INFO['AS'],
    m_nu = COSMOLOGY_INFO['M_NU'],  
    Neff = COSMOLOGY_INFO['N_EFF'],
    Omega_b = COSMOLOGY_INFO['OMEGA_B'], 
    Omega_k = COSMOLOGY_INFO['OMEGA_K'], 
    Omega_c = COSMOLOGY_INFO['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

Calculate the power spectrum.

In [ ]:
# Define the redshift grid
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

# Calculate corresponding scale factors and comoving distances
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Calculate the power spectrum on the grid
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=COSMOLOGY, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

In [5]:
# Numerical integral

def integral_I9(chi1, chi2, chi3, chi4, chi5, chi6, chi7, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        result = ((chi5 - chi3) / 2 - chi * chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4) + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3))
        
        result = result * (- chi + (chi7 - chi6) / 2 + chi * chi6 * numpy.log(chi7 / chi6) / (chi7 - chi6))
        
        result = result * (((chi / chi2) ** 3) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    def formula_n(chi):
        result = ((chi5 - chi3) / 2 - chi * chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4) + chi * chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3))
        
        result = result * (- chi + (chi7 - chi6) / 2 + chi * chi6 * numpy.log(chi7 / chi6) / (chi7 - chi6))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

In [6]:
# Coefficient

def coefficient_J9(chi1, chi2, chi3, chi4, chi5, chi6, chi7, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = (chi5 - chi3) / (2 * chi2)
    c = chi3 * numpy.log(chi4 / chi3) / (chi4 - chi3) - chi5 * numpy.log(chi5 / chi4) / (chi5 - chi4)
    d = (chi7 - chi6) / (2 * chi2)
    e = chi6 * numpy.log(chi7 / chi6) / (chi7 - chi6) - 1
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (1 / 840) * (5 * c * e * (28 + ( - 8 + z) * z) + 8 * c * d * (21 + ( - 7 + z) * z) + 8 * b * e * (21 + ( - 7 + z) * z) + 14 * b * d * (15 + ( - 6 + z) * z))
    else:
        formula = (1 / 60) * a * (b * ( - 30 * e * ( - 2 + p + 2 * z) + 5 * e * z * (p * (8 - 3 * z) + 4 * z) + 20 * d * (3 + ( - 3 + z) * z) + a * e * ( - 30 + 20 * p + 5 * (8 - 3 * z) * z + 6 * p * z * ( - 5 + 2 * z)) - 
5 * d * p * (6 + z * ( - 8 + 3 * z))) + c * (e * (20 * (3 + ( - 3 + a) * a) - 5 * (6 + a * ( - 8 + 3 * a)) * p + 2 * ( - 30 + 5 * (8 - 3 * a) * a + 20 * p + 6 * a * ( - 5 + 2 * a) * p) * z + 
(20 - 15 * p + 2 * a * ( - 15 + 6 * a + 12 * p - 5 * a * p)) * z ** 2) + d * (20 * (3 + ( - 3 + z) * z) + a * ( - 30 + 20 * p + 5 * (8 - 3 * z) * z + 6 * p * z * ( - 5 + 2 * z)) - 
5 * p * (6 + z * ( - 8 + 3 * z)))))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])
CHI3 = numpy.float128(CHI_GRID[GRID_INDEX + 2])
CHI4 = numpy.float128(CHI_GRID[GRID_INDEX + 3])
CHI5 = numpy.float128(CHI_GRID[GRID_INDEX + 4])
CHI6 = numpy.float128(CHI_GRID[GRID_SIZE - 1])
CHI7 = numpy.float128(CHI_GRID[GRID_SIZE])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 4957.1305889213954288 4973.4637436395141776 4989.717316369170476 6946.038887321783477 6954.2547327845313703 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

226965.16102398655164 226965.16102398493868 -7.1066199799907359136e-15


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_INDEX + 3]
CHI5 = CHI_GRID[GRID_INDEX + 4]
CHI6 = CHI_GRID[GRID_SIZE - 1]
CHI7 = CHI_GRID[GRID_SIZE]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 88.58943766284152 132.5655204109753 176.3277722724983 6946.0388873217835 6954.254732784531 0.0 1826.8838600584127 0.0 0.01


In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J9(CHI1, CHI2, CHI3, CHI4, CHI5, CHI6, CHI7, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

2684401.9789093924 2684401.9789093905 -6.661338147750939e-16
